In [1]:
import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")
#Pkg.add("PyCall")

In [2]:
using JuMP, GLPK
using DataFrames
using CSV

In [3]:
# Load the data
scenarios_df = CSV.read("../data/scenarios.csv", DataFrame)

n_scenarios = size(scenarios_df, 2)/3
n_scenarios = convert(Int, n_scenarios)

# create a dictonary with 200 dataframes for each scenario
scenarios = Dict()
for i in 1:n_scenarios
    df_helper = DataFrame(scenarios_df[:,3*i-2:3*i])
    df_helper[!,3] = df_helper[!,3] .* 1.0
    rename!(df_helper, [:"price", :"wind power", :"grid_excess"])
    scenarios[i] = df_helper
end

In [4]:
scenarios[3]

Row,price,wind power,grid_excess
,Float64,Float64,Float64
1,2.01,114.776,1.0
2,1.38,117.266,0.0
3,0.09,117.017,1.0
4,0.08,113.003,1.0
5,0.05,108.147,0.0
6,0.0,109.435,1.0
7,0.0,104.349,1.0
8,0.0,103.626,1.0
9,0.0,103.57,0.0


In [5]:
#scenarios[2]

In [6]:
# Initialize the DataFrame directly without dynamic column names
result_df = DataFrame(hour = Int[], p_DA = Float64[], delta = Float64[], delta_up = Float64[], delta_down = Float64[])

Row,hour,p_DA,delta,delta_up,delta_down
,Int64,Float64,Float64,Float64,Float64


In [7]:
W = 250
hours = 24

24

In [8]:
# Create a new model with GLPK solver
model = Model(GLPK.Optimizer)

# Define the decision variables for hour
@variable(model, p_DA[1:hours])
@variable(model, delta[1:W,1:hours])
@variable(model, delta_up[1:W,1:hours])
@variable(model, delta_down[1:W,1:hours])

# Define the objective function
@objective(model, Max, sum(1/W*(
        scenarios[i][hour,"price"] * p_DA[hour]
     + delta_up[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*0.9 + (1-scenarios[i][hour,"grid_excess"])*1)
      - delta_down[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*1 + (1-scenarios[i][hour,"grid_excess"])*1.2)
      ) for i in 1:W, hour in 1:hours))

# Define the constraints
@constraint(model, [hour in 1:hours], p_DA[hour] <= 200)
@constraint(model, [hour in 1:hours], p_DA[hour] >= 0)
@constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == scenarios[i][hour,"wind power"] - p_DA[hour])
@constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == delta_up[i,hour] - delta_down[i,hour])
@constraint(model, [i in 1:W, hour in 1:hours], delta_down[i,hour] >= 0)
#@constraint(model, [i in 1:W, hour in 1:hours], delta_down[i,hour] <= p_DA[hour])
@constraint(model, [i in 1:W, hour in 1:hours], delta_up[i,hour] >= 0)
#@constraint(model, [i in 1:W, hour in 1:hours], delta_up[i,hour] + p_DA[hour] <= 200)



# Solve the optimization problem
optimize!(model)


#println(model)

# Print the termination status
status = termination_status(model)
if status == MOI.OPTIMAL
    println("Optimal solution found")
    
    # RETURN OBJECTIVE value
    println("Objective value: ", objective_value(model))
else
    println("No optimal solution found")
end

Optimal solution found
Objective value: 173475.3995243449


In [9]:
println("Objective value: ", objective_value(model))
println("p_DA: ", value.(p_DA))
println("delta: ", value.(delta))
println("delta_up: ", value.(delta_up))
println("delta_down: ", value.(delta_down))

Objective value: 173475.3995243449
p_DA: [64.39741935483872, 14.389693451612905, 56.90564516129032, 50.04241930645161, 46.991290322580646, 45.95790322580645, 145.9269354516129, 13.34106441935484, 1.1038967741935486, 146.94499998387096, 22.81691932258065, 0.7419306451612904, 142.91516125806453, 50.27709674193549, 135.9887096612903, 41.58451606451612, 146.0248386935484, 149.60145158064518, 27.48990317741936, 152.74596772580645, 4.493516129032258, 75.83806443548387, 6.808774177419355, 149.57870966129033]
delta: [50.37887096774192 102.87611298387098 60.11129032258066 62.960483887096764 61.155322532258076 63.47693546774195 -41.577741919354864 90.28474201612904 102.46594193548385 -42.06048387096776 87.30920967741936 109.49935967741935 -37.72919354838713 62.11483874193548 -19.907419370967716 71.41225811290323 -31.801612887096795 -32.817419354838734 91.0678386935484 -36.46483874193545 103.23390320967742 30.63451620967743 98.32654840322583 -44.11983870967741; 50.37887096774192 102.8761129838709

In [10]:
#make a table with the results
scenario = 1
for hour in 1:hours
    push!(result_df, [hour, value(p_DA[hour]), value(delta[scenario,hour]), value(delta_up[scenario,hour]), value(delta_down[scenario,hour])])
end

result_df

Row,hour,p_DA,delta,delta_up,delta_down
,Int64,Float64,Float64,Float64,Float64
1,1,64.3974,50.3789,50.3789,0.0
2,2,14.3897,102.876,102.876,0.0
3,3,56.9056,60.1113,60.1113,0.0
4,4,50.0424,62.9605,62.9605,0.0
5,5,46.9913,61.1553,61.1553,0.0
6,6,45.9579,63.4769,63.4769,0.0
7,7,145.927,-41.5777,0.0,41.5777
8,8,13.3411,90.2847,90.2847,0.0
9,9,1.1039,102.466,102.466,0.0
